In [1]:
# Library Import

import os
import sys

In [2]:
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment'

In [ ]:
# perparing the data entities [config_entity]

from dataclasses import dataclass
from pathlib import Path

# Below is the data class for data ingestion configuration
# This class is used to define the configuration for data ingestion in a machine learning pipeline.
@dataclass(frozen=True)
class DataIngestionConfig:
    """Data Ingestion Configuration"""
    # Define the attributes of the class
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    """Configuration Manager"""
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        """Initialize Configuration Manager"""
        # Initialize the ConfigurationManager with file paths for configuration, parameters, and schema.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)



        # Create directories for artifacts and data ingestion
        create_directories([self.config.artifacts_root])  # Create the artifacts root directory

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Get Data Ingestion Configuration"""
        config = self.config.data_ingestion
        create_directories([config.root_dir])  # Create the data ingestion root directory

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from ml_project import logger
from ml_project.utils.common import get_size

In [25]:
class DataIngestion:
    """Data Ingestion"""
    def __init__(self, config: DataIngestionConfig):
        """Initialize Data Ingestion"""
        self.config = config


    def download_file(self):
        """Download file from URL"""
        if not os.path.exists(self.config.local_data_file):

            # Download the file from the source URL to the local data file path
            request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"Downloading file from {self.config.source_URL} to {self.config.local_data_file}")
            logger.info(f"Downloaded file size: {get_size(Path(self.config.local_data_file))} bytes")
        else:
            logger.info(f"File already exists at {self.config.local_data_file} of size: {get_size(Path(self.config.local_data_file))} bytes. Skipping download.")


    def extract_zip_file(self):
            
        """Extract zip file"""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted zip file to {unzip_path}")
        return unzip_path


In [27]:
try:
    """from ml_project import logger
    from ml_project.config.configuration import ConfigurationManager
    from ml_project.components.data_ingestion import DataIngestion"""

    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-30 00:28:07,277: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-30 00:28:07,279: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-30 00:28:07,282: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-30 00:28:07,285: INFO: common: created directory at: artifacts]
[2025-04-30 00:28:07,287: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-30 00:28:07,290: INFO: 3877789902: File already exists at artifacts/data_ingestion/data.zip of size: ~ 23 KB bytes. Skipping download.]
[2025-04-30 00:28:07,297: INFO: 3877789902: Extracted zip file to artifacts/data_ingestion]
